In [5]:
import numpy as np
import scipy.signal

# Calculate velocity correlations

In [8]:
def extract_vel(lines,i1,i2):
    v1 = []
    v2 = []
    for i in range(len(lines)):
        if lines[i]=='ITEM: NUMBER OF ATOMS\n':
            natom = int(lines[i+1].split()[0])
        if lines[i]=='ITEM: ATOMS element vx vy vz\n':
            x1 = float(lines[i+i1+1].split()[1])
            y1 = float(lines[i+i1+1].split()[2])
            z1 = float(lines[i+i1+1].split()[3])
            
            x2 = float(lines[i+i2+1].split()[1])
            y2 = float(lines[i+i2+1].split()[2])
            z2 = float(lines[i+i2+1].split()[3])
            
            v1.append([x1,y1,z1])
            v2.append([x2,y2,z2])
    return np.array(v1),np.array(v2)

In [9]:
def cond_lag_lim_vec_fast(vel_chunk_1,vel_chunk_2,tstep,samp_freq,lag_len):
    
    if len(vel_chunk_1)!=len(vel_chunk_2):    #check if both velocity chunks have same size
        print("Error")

    #sampling of charge current every lag_len timeteps    
    
    inte = []
    shift = lag_len/samp_freq
    a1 = np.arange(0,vel_chunk_1.shape[0],shift)
    a2 = np.arange(0,vel_chunk_2.shape[0],shift)
    
    v_x1 = np.array([vel_chunk_1[int(i)][0] for i in a1])
    v_y1 = np.array([vel_chunk_1[int(i)][1] for i in a1])
    v_z1 = np.array([vel_chunk_1[int(i)][2] for i in a1])
    v_x2 = np.array([vel_chunk_2[int(i)][0] for i in a2])
    v_y2 = np.array([vel_chunk_2[int(i)][1] for i in a2])
    v_z2 = np.array([vel_chunk_2[int(i)][2] for i in a2])

    #calculate correlation between two chunks
    
    acf_temp_x = scipy.signal.correlate(v_x1,v_x2)
    acf_temp_y = scipy.signal.correlate(v_y1,v_y2)
    acf_temp_z = scipy.signal.correlate(v_z1,v_z2)
    
    acf_x = np.flip(acf_temp_x[0:len(v_x1)])
    acf_y = np.flip(acf_temp_y[0:len(v_y1)])
    acf_z = np.flip(acf_temp_z[0:len(v_z1)])
    
    acf_x_rev = acf_temp_x[len(v_x1)-1:]
    acf_y_rev = acf_temp_y[len(v_y1)-1:]
    acf_z_rev = acf_temp_z[len(v_z1)-1:]
    acf_tot = (acf_x + acf_y + acf_z + acf_x_rev + acf_y_rev + acf_z_rev)/(2*3)
    
    norm = np.flip(np.arange(1,len(v_x1)+1,1))
    acf_norm = acf_tot/norm
    
    for i in range(len(acf_tot)):
        temp = np.trapz(acf_norm[0:i+1],dx=(lag_len*tstep))
        inte.append(temp)
        
    return acf_norm,inte

In [10]:
filename = '/Users/rajorshi/Desktop/test/NVT_prod_vel_ion.lammpstrj'

with open(filename) as f:
    lines = f.readlines()

#number of cations and anions

n_cat = 4
n_an = 4

#create list containing ions in order as they appear in trajectory file

ion = ['Na' for _ in range(n_cat)]
ion.extend(['Cl' for _ in range(n_an)])

    
samp_freq = 10          #sampling frequency in timesteps
lag_len = 10            #timesteps. Has to be multiple of sampling frequency
tstep = 0.5            #timestep of simulation
T = 1073

#counters to count ion pairs

count_cat_cat_self = 0
count_an_an_self = 0
count_cat_cat_cross = 0
count_an_an_cross = 0
count_cat_an = 0

for i1 in range(len(ion)):
    for i2 in np.arange(i1,len(ion),1):
        vel_chunk_1,vel_chunk_2 = extract_vel(lines,int(i1),int(i2))
        acf_tot,inte = cond_lag_lim_vec_fast(vel_chunk_1,vel_chunk_2,tstep,samp_freq,lag_len)
        
        if ion[i1]=='Na' and ion[i2]=='Na' and i1==i2:    #cation-cation self
            count_cat_cat_self = count_cat_cat_self + 1  
            if count_cat_cat_self==1:
                corr_cat_cat_self = np.zeros((len(vel_chunk_1)))
                inte_cat_cat_self = np.zeros((len(vel_chunk_1)))
                corr_cat_cat_self = corr_cat_cat_self + acf_tot
                inte_cat_cat_self = inte_cat_cat_self + inte
            else:
                corr_cat_cat_self = corr_cat_cat_self + acf_tot
                inte_cat_cat_self = inte_cat_cat_self + inte
                
        if ion[i1]=='Na' and ion[i2]=='Na' and i1!=i2:    #cation-cation cross
            count_cat_cat_cross = count_cat_cat_cross + 1
            if count_cat_cat_cross==1:
                corr_cat_cat_cross = np.zeros((len(vel_chunk_1)))
                inte_cat_cat_cross = np.zeros((len(vel_chunk_1)))
                corr_cat_cat_cross = corr_cat_cat_cross + acf_tot
                inte_cat_cat_cross = inte_cat_cat_cross + inte
            else:
                corr_cat_cat_cross = corr_cat_cat_cross + acf_tot
                inte_cat_cat_cross = inte_cat_cat_cross + inte
                    
        if ion[i1]=='Cl' and ion[i2]=='Cl' and i1==i2:    #anion-anion self
            count_an_an_self = count_an_an_self + 1
            if count_an_an_self==1:
                corr_an_an_self = np.zeros((len(vel_chunk_1)))
                inte_an_an_self = np.zeros((len(vel_chunk_1)))
                corr_an_an_self = corr_an_an_self + acf_tot
                inte_an_an_self = inte_an_an_self + inte
            else:
                corr_an_an_self = corr_an_an_self + acf_tot
                inte_an_an_self = inte_an_an_self + inte
                
        if ion[i1]=='Cl' and ion[i2]=='Cl' and i1!=i2:     #anion-anion cross
            count_an_an_cross = count_an_an_cross + 1
            if count_an_an_cross==1:
                corr_an_an_cross = np.zeros((len(vel_chunk_1)))
                inte_an_an_cross = np.zeros((len(vel_chunk_1)))
                corr_an_an_cross = corr_an_an_cross + acf_tot
                inte_an_an_cross = inte_an_an_cross + inte
            else:
                corr_an_an_cross = corr_an_an_cross + acf_tot
                inte_an_an_cross = inte_an_an_cross + inte
                    
        if (ion[i1]=='Na' and ion[i2]=='Cl') or (ion[i1]=='Cl' and ion[i2]=='Na'):   #cation-anion
            count_cat_an = count_cat_an + 1
            if count_cat_an==1:
                corr_cat_an = np.zeros((len(vel_chunk_1)))
                inte_cat_an = np.zeros((len(vel_chunk_1)))
                corr_cat_an = corr_cat_an + acf_tot
                inte_cat_an = inte_cat_an + inte
            else:
                corr_cat_an = corr_cat_an + acf_tot
                inte_cat_an = inte_cat_an + inte 

In [11]:
filename_prog_self = '/Users/rajorshi/Desktop/test/self_corr_rev.out'
filename_prog_cross = '/Users/rajorshi/Desktop/test/cross_corr_rev.out'

f_prog_self = open(filename_prog_self,'w')
f_prog_cross = open(filename_prog_cross,'w')

f_prog_self.write('Timestep: '+str(tstep)+'\n')
f_prog_self.write("Temp: "+str(T)+'\n')

f_prog_cross.write('Timestep: '+str(tstep)+'\n')
f_prog_cross.write("Temp: "+str(T)+'\n')

#calculate correlation contribution for each ion/ion pair

corr_cat_cat_self = corr_cat_cat_self/count_cat_cat_self
corr_cat_cat_cross = corr_cat_cat_cross/count_cat_cat_cross
corr_an_an_self = corr_an_an_self/count_an_an_self
corr_an_an_cross = corr_an_an_cross/count_an_an_cross
corr_cat_an = corr_cat_an/count_cat_an

inte_cat_cat_self = inte_cat_cat_self/count_cat_cat_self
inte_cat_cat_cross = inte_cat_cat_cross/count_cat_cat_cross
inte_an_an_self = inte_an_an_self/count_an_an_self
inte_an_an_cross = inte_an_an_cross/count_an_an_cross
inte_cat_an = inte_cat_an/count_cat_an

#print out data in file

f_prog_self.write("Cation-Cation\n")
f_prog_self.write('Lag'+'    '+'Lag (in ps)'+'     '+'Auto-correlation'+'     '+'Integral(acf)'+'\n')

lag = 0       
    
for i in range(len(corr_cat_cat_self)):
    f_prog_self.write(str(lag)+'      '+str((lag*tstep)/1000)+'          '+str(corr_cat_cat_self[i])+'    '+str(inte_cat_cat_self[i])+'\n') 
    lag = lag + lag_len

f_prog_self.write("Anion-Anion\n")
f_prog_self.write('Lag'+'    '+'Lag (in ps)'+'     '+'Auto-correlation'+'     '+'Integral(acf)'+'\n')
lag = 0 

for i in range(len(corr_an_an_self)):
    f_prog_self.write(str(lag)+'      '+str((lag*tstep)/1000)+'          '+str(corr_an_an_self[i])+'    '+str(inte_an_an_self[i])+'\n') 
    lag = lag + lag_len

f_prog_cross.write("Cation-Cation\n")
f_prog_cross.write('Lag'+'    '+'Lag (in ps)'+'     '+'Auto-correlation'+'     '+'Integral(acf)'+'\n')

lag = 0 
for i in range(len(corr_cat_cat_cross)):
    f_prog_cross.write(str(lag)+'      '+str((lag*tstep)/1000)+'          '+str(corr_cat_cat_cross[i])+'    '+str(inte_cat_cat_cross[i])+'\n') 
    lag = lag + lag_len

f_prog_cross.write("Anion-Anion\n")
f_prog_cross.write('Lag'+'    '+'Lag (in ps)'+'     '+'Auto-correlation'+'     '+'Integral(acf)'+'\n')

lag = 0 
for i in range(len(corr_an_an_cross)):
    f_prog_cross.write(str(lag)+'      '+str((lag*tstep)/1000)+'          '+str(corr_an_an_cross[i])+'    '+str(inte_an_an_cross[i])+'\n') 
    lag = lag + lag_len
    
f_prog_cross.write("Cation-Anion\n")
f_prog_cross.write('Lag'+'    '+'Lag (in ps)'+'     '+'Auto-correlation'+'     '+'Integral(acf)'+'\n')

lag = 0 
for i in range(len(corr_cat_an)):
    f_prog_cross.write(str(lag)+'      '+str((lag*tstep)/1000)+'          '+str(corr_cat_an[i])+'    '+str(inte_cat_an[i])+'\n') 
    lag = lag + lag_len


#f_prog.write('Total time:'+' '+str(wall_time))
f_prog_self.close()
f_prog_cross.close()

In [13]:
#some important stuff

kB = 1.3806504e-23       #in J/K
T = 1073                 
box_max = 20.70783
box_min = 0.99219617
V = (box_max-box_min)**3

#charge of ions

q_an = -1       
q_cat = 1

n_cat_cat_cross = 6   #number of distinct cation-cation pairs for cross correlation
n_an_an_cross = 6     #number of distinct anion-anion pairs for cross correlation
n_cat_an = 16         #number of distinct cation-anion pairs for cross correlation
n_cat = 4             #number of distinct cations for self correlation
n_an = 4              #number of distinct anions for self correlation

#unit conversions from LAMMPS real units to SI units

ang2m = 1*10**-10
e2coul = 1.6021*10**-19
fs2s = 1*10**-15
vel_conv = ang2m/fs2s
conv = (((ang2m/fs2s)**2)*fs2s*(e2coul**2))/(ang2m**3)

# Cross correlation contributions

In [14]:
filename = '/Users/rajorshi/Desktop/test/cross_corr_rev.out'

with open(filename) as f:
    lines = f.readlines()
start = []
end = []
for i in range(len(lines)):   #extract data from velocity correlation file
    s = lines[i].split()
    if s[0] == 'Lag':
        start.append(i+1)
    elif s[0] == 'Anion-Anion' or s[0] == 'Cation-Anion':
        end.append(i)

end.append(len(lines))

time_lim = 1                 #integration time of velocity correlation functions in ps
data_acf_cat_cat_cross = []
lag_time = []
data_acf_norm_cat_cat_cross = []
inte_cat_cat_cross = []

data_acf_an_an_cross = []
data_acf_norm_an_an_cross = []
inte_an_an_cross = []

data_acf_cat_an_cross = []
data_acf_norm_cat_an_cross = []
inte_cat_an_cross = []

for i in lines[start[0]:end[0]]:
    if float(i.split()[1]) <= time_lim:
        data_acf_cat_cat_cross.append(float(i.split()[2]))
        lag_time.append(float(i.split()[1]))
        inte_cat_cat_cross.append(float(i.split()[3]))
        
for i in lines[start[1]:end[1]]:
    if float(i.split()[1]) <= time_lim:
        data_acf_an_an_cross.append(float(i.split()[2]))
        inte_an_an_cross.append(float(i.split()[3]))
        
for i in lines[start[2]:end[2]]: 
    if float(i.split()[1]) <= time_lim:
        data_acf_cat_an_cross.append(float(i.split()[2]))
        inte_cat_an_cross.append(float(i.split()[3]))
        
data_acf_norm_cat_cat_cross = [data_acf_cat_cat_cross[i]/data_acf_cat_cat_cross[0] for i in range(len(data_acf_cat_cat_cross))]
data_acf_norm_an_an_cross = [data_acf_an_an_cross[i]/data_acf_an_an_cross[0] for i in range(len(data_acf_an_an_cross))]
data_acf_norm_cat_an_cross = [data_acf_cat_an_cross[i]/data_acf_cat_an_cross[0] for i in range(len(data_acf_cat_an_cross))]

In [15]:
#total contribution from all ion pairs

inte_an_an_cross = np.array(inte_an_an_cross)*n_an_an_cross    
inte_cat_cat_cross = np.array(inte_cat_cat_cross)*n_cat_cat_cross
inte_cat_an_cross = np.array(inte_cat_an_cross)*n_cat_an

#conductivity contribution

cond_cat_cat_cross = (conv* inte_cat_cat_cross * (q_cat*q_cat))/(kB*T*V)
cond_an_an_cross = (conv* inte_an_an_cross * (q_an*q_an))/(kB*T*V)
cond_cat_an_cross = (conv* inte_cat_an_cross * (q_cat*q_an))/(kB*T*V)

# Self correlation contribution 

In [16]:
filename = '/Users/rajorshi/Desktop/test/self_corr_rev.out'

with open(filename) as f:
    lines = f.readlines()
start = []
end = []
for i in range(len(lines)):
    s = lines[i].split()
    if s[0] == 'Lag':
        start.append(i+1)
    elif s[0] == 'Anion-Anion':
        end.append(i)
        
end.append(len(lines))

time_lim = 1           #integration time of velocity correlation functions in ps
data_acf_cat_cat_self = []
lag_time = []
data_acf_norm_cat_cat_self = []
inte_cat_cat_self = []

data_acf_an_an_self = []
data_acf_norm_an_an_self = []
inte_an_an_self = []

for i in lines[start[0]:end[0]]:
    if float(i.split()[1]) <= time_lim:
        data_acf_cat_cat_self.append(float(i.split()[2]))
        lag_time.append(float(i.split()[1]))
        inte_cat_cat_self.append(float(i.split()[3]))
        
for i in lines[start[1]:end[1]]:
    if float(i.split()[1]) <= time_lim:
        data_acf_an_an_self.append(float(i.split()[2]))
        inte_an_an_self.append(float(i.split()[3]))
        
        
data_acf_norm_cat_cat_self = [data_acf_cat_cat_self[i]/data_acf_cat_cat_self[0] for i in range(len(data_acf_cat_cat_self))]
data_acf_norm_an_an_self = [data_acf_an_an_self[i]/data_acf_an_an_self[0] for i in range(len(data_acf_an_an_self))]

In [17]:
#contribution of all ions

inte_an_an_self = np.array(inte_an_an_self)
inte_cat_cat_self = np.array(inte_cat_cat_self)

#conductivity contribution
inte_tot_self = inte_an_an_self+inte_cat_cat_self
cond_self_an = (conv* inte_an_an_self * (q_an**2)*n_an)/(kB*T*V)
cond_self_cat = (conv* inte_cat_cat_self * (q_cat**2)*n_cat)/(kB*T*V)

In [18]:
#print out spectral decomposition data

filename = '/Users/rajorshi/Desktop/test/cond_spec_rev.out'

f = open(filename,'w')

f.write('Box size: max: '+str(box_max)+' min: '+str(box_min)+' in Ang'+'\n')
f.write('T: '+str(T)+'\n')
f.write('Cation charge: '+str(q_cat)+'  '+'Anion charge: '+str(q_an)+'\n')

f.write('Lag time'+'  '+'Cat-Cat_self'+'  '+'An-An_self'+'  '+'Cat-Cat_cross'+'  '+'An-An_cross'+'  '+'Cat-An_cross'+'\n')

for i in range(len(lag_time)):
    f.write(str(lag_time[i])+'  '+str(cond_self_cat[i])+'  '+str(cond_self_an[i])+'  '+str(cond_cat_cat_cross[i])+'  '+str(cond_an_an_cross[i])+'  '+str(cond_cat_an_cross[i]*2)+'\n')

f.close()


In [19]:
#print self diffusion constant of ions

filename = '/Users/rajorshi/Desktop/test/diff_const_rev.out'

conv_disp = (vel_conv**2)*fs2s

f = open(filename,'w')

f.write('Box size: max: '+str(box_max)+' min: '+str(box_min)+' in Ang'+'\n')
f.write('T: '+str(T)+'\n')
f.write('Cation charge: '+str(q_cat)+'  '+'Anion charge: '+str(q_an)+'\n')

f.write('Lag time'+'  '+'Cat'+'  '+'An'+'\n')

for i in range(len(lag_time)):
    f.write(str(lag_time[i])+'  '+str(inte_cat_cat_self[i]*conv_disp)+'  '+str(inte_an_an_self[i]*conv_disp)+'\n')

f.close()    